In [1]:
from pdf2image import convert_from_path

In [44]:
pages = convert_from_path("resources\patient_details\pd_2.pdf",poppler_path=r'C:\Release-24.02.0-0\poppler-24.02.0\Library\bin')

In [45]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1500x2000 at 0x21280EF60D0>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1992x2000 at 0x21280EF63A0>]

In [46]:
# pages[0].show()

In [47]:
import pytesseract

In [48]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(pages[0],lang='eng')
print(text)

a

Patient Medical Record

Patient Information
Jerry Lucas

(279) 920-8204
4218 Wheeler Ridge Dr

Buffalo, New York, 14201
United States

In Case of Emergency

Birth Date
May 2 1998

Weight:
57

Height:
170

Joe Lucas

Home phone

General Medical History

4218 Wheeler Ridge Dr
Buffalo, New York, 14201
United States

Work phone

Chicken Pox (Varicelia):

IMMUNE

Have you had the Hepatitis B vaccination?

Measles:

NOT IMMUNE

any Medical Problems (asthma, seizures, headaches):


In [59]:
import numpy as np
import cv2
from PIL import Image
def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img),cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray,None,fx=2.1,fy=2.1,interpolation=cv2.INTER_LINEAR)
    procesed_image = cv2.adaptiveThreshold(
        resized_img,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        61,
        11 
    )
    return procesed_image



In [60]:
img = preprocess_image(pages[0])
Image.fromarray(img).show()

In [61]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(img,lang='eng')
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date

Jerry Lucas May 2 1998

(279) 920-8204 Weight:

4218 Wheeler Ridge Dr 97

Buffalo, New York, 14201 Height:

United States ; gn
170

In Case of Emergency ,
es

Joe Lucas . 4218 Wheeler Ridge Dr
Buffalo, New York, 14201
Home phone . United States
Work phone

General Medical History

Chicken Pox (Varicelia): Measles: |
IMMUNE NOT IMMUNE

Have you had the Hepatitis B vaccination?
Yes” |

__ List any Medical Problems (asthma, seizures, headaches):
N/A


### Extract name

In [30]:
import re

pattern = 'Patient Information(.*?)\(\d{3}\)'

matches = re.findall(pattern, text,flags=re.DOTALL)
matches[0].strip()

'Birth Date\nKathy Crawford May 6 1972'

In [21]:
match = matches[0].replace("Birth Date","").strip()
match

'Kathy Crawford May 6 1972'

In [26]:
pattern = '((Jan|Feb|MaApr|May|Jun|Jul|Aug) [ \d]+)'
date_matches = re.findall(pattern,match)
date = date_matches[0][0]

In [27]:
match.replace(date,"").strip()

'Kathy Crawford'

In [35]:
def remove_noise_from_name(name):
    match = matches[0].replace("Birth Date","").strip()
    pattern = '((Jan|Feb|MaApr|May|Jun|Jul|Aug) [ \d]+)'
    date_matches = re.findall(pattern,match)

    if date_matches:
        date = date_matches[0][0]
        match = match.replace(date,"").strip()
    
    return match

In [36]:
name = matches

name = remove_noise_from_name(name)
name

'Kathy Crawford'

### Extract phone

In [38]:
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'

matches = re.findall(pattern, text,flags=re.DOTALL)
matches[0]

('\n\nBirth Date\nKathy Crawford May 6 1972\n', '(737) 988-0851')

In [40]:
matches[0][1]

'(737) 988-0851'

### Extract Vaccine

In [41]:
pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'

matches = re.findall(pattern, text,flags=re.DOTALL)
matches[0]

'No'

### Extract medical problems

In [43]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\):(.*)'

matches = re.findall(pattern, text,flags=re.DOTALL)
matches[0].strip()

'Migraine'